# Configuração e Carregamento de Dados

In [5]:
!pip install pyspark # Instala a biblioteca PySpark no ambiente de execução

In [6]:
import pyspark # mporta a biblioteca principal do PySpark
from pyspark.sql import SparkSession # Importa a classe SparkSession, o ponto de entrada para o Spark

In [7]:
from google.colab import drive #  Importa o módulo drive do Google Colab
drive.mount('/content/drive') # Monta o Google Drive para acesso aos arquivos

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
spark = SparkSession.builder.getOrCreate() #  Cria ou obtém a instância da SparkSession

In [9]:
# Lê todos os DataFrames do caminho especificado
produtos = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/produtos.csv", header=True, inferSchema=True)
vendedores  = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/vendedores.csv", header=True, inferSchema=True)
clientes  = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/clientes.csv", header=True, inferSchema=True)
itens_pedido  = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/itens_pedido.csv", header=True, inferSchema=True)
pagamentos_pedido = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/pagamentos_pedido.csv", header=True, inferSchema=True)
avaliacoes_pedido  = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/avaliacoes_pedido.csv", header=True, inferSchema=True)
pedidos  = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/pedidos.csv", header=True, inferSchema=True)

In [10]:
produtos.show() # Exibe as primeiras linhas do DataFrame produtos

+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|          id_produto|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|1e9e8ef04dbcff454...|          perfumaria|                  40|                      287|                       1|           225|                    16|               10|                14|
|3aa071139cb16b67c...|               artes|                  44|                      276|                       1|          1000|                    30|               18|                20|
|96bd76ec8810374ed...|       esporte_lazer|  

# Manipulação de Particionamento (RDD)

In [11]:
rdd = spark.sparkContext.parallelize(range(100)) # Cria um RDD a partir de uma lista de números (0 a 99). O Spark atribui um número de partições padrão
print("número padrão de partições:", rdd.getNumPartitions()) # Exibe o número de partições padrão (geralmente 4, 8 ou o número de cores da máquina virtual)

número padrão de partições: 2


In [12]:
rdd = spark.sparkContext.parallelize(range(100),10) # Cria o mesmo RDD, mas forçando 10 partições
print("número de partições após modificação:", rdd.getNumPartitions()) # Confirma que o número de partições é 10

número de partições após modificação: 10


In [13]:
reduced_rdd = rdd.coalesce(5) # coalesce (Diminuir): Reduz o número de partições de 10 para 5. coalesce evita o shuffle de dados (movimentação cara de dados), sendo ideal para diminuir o número de partições de forma eficiente

print("Número de partições após coalesce:", reduced_rdd.getNumPartitions()) # Confirma que o número de partições é 5

Número de partições após coalesce: 5


In [14]:
repartitioned_rdd = reduced_rdd.repartition(20) # repartition (Aumentar): Altera o número de partições de 5 para 20. repartition sempre realiza um shuffle de dados, sendo adequado tanto para aumentar quanto para diminuir o número de partições, garantindo uma distribuição mais uniforme, mas com custo
print("número de partições após repartition:", repartitioned_rdd.getNumPartitions()) # Confirma que o número de partições é 20

número de partições após repartition: 20


# Manipulação de Particionamento (DataFrame)

In [15]:
print("número de partições padrão de produtos:", produtos.rdd.getNumPartitions()) # Exibe o número de partições padrão do DataFrame produtos (herdado da leitura do CSV).

produtos = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/produtos.csv", header=True, inferSchema=True).repartition(5) # Lê o DataFrame e, em seguida, força um repartition(5). Isso fará com que o Spark execute um shuffle inicial para ter 5 partições
print("número de partições com repartition de produtos:", produtos.rdd.getNumPartitions()) # Confirma que o número de partições é 5

número de partições padrão de produtos: 1
número de partições com repartition de produtos: 5


In [16]:
produtos.groupBy("categoria_produto").count().show() # Agrupa e conta a frequência de cada categoria
print(produtos.select("categoria_produto").distinct().count()) # Conta o número de categorias de produto distintas

produtos_part_categoria = produtos.repartition("categoria_produto") # Realiza um repartition (shuffle) usando a coluna categoria_produto como chave. O número de partições será o número padrão (geralmente o padrão do Spark, como 200)
print(produtos_part_categoria.rdd.getNumPartitions()) # Exibe o número de partições padrão após o repartition por chave

+--------------------+-----+
|   categoria_produto|count|
+--------------------+-----+
|                 pcs|   30|
|               bebes|  919|
|           cine_foto|   28|
|               artes|   55|
|    moveis_decoracao| 2657|
|construcao_ferram...|  400|
|tablets_impressao...|    9|
|   artigos_de_festas|   26|
|fashion_roupa_mas...|   95|
|    artigos_de_natal|   65|
|          la_cuisine|   10|
|              flores|   14|
|     livros_tecnicos|  123|
|                NULL|  610|
|      telefonia_fixa|  116|
|construcao_ferram...|   91|
|          cool_stuff|  789|
|    eletrodomesticos|  370|
|   livros_importados|   31|
|            pet_shop|  719|
+--------------------+-----+
only showing top 20 rows

74
1


In [17]:
produtos_part_categoria = produtos.repartition(8, "categoria_produto") # Realiza um repartition usando a coluna categoria_produto, mas força o número de partições para 8. Isso é útil para otimizar o paralelismo
print(produtos_part_categoria.rdd.getNumPartitions()) # Confirma que o número de partições é 8

particoes = produtos_part_categoria.rdd.glom().map(len).collect() # Coleta o número de registros (tamanho) de cada partição para verificar a distribuição
print("número de registros por partição", particoes) # Exibe o tamanho de cada uma das 8 partições

8
número de registros por partição [6626, 1298, 5302, 6406, 5322, 3379, 692, 3926]


# Persistência com Diferentes Estratégias de Particionamento

In [18]:
# Salva o DataFrame produtos (que tinha 5 partições) no diretório especificado. Serão criados 5 arquivos Parquet (um por partição)
produtos.write.mode("overwrite").parquet("produtos_default_parquet")

In [19]:
# Usa coalesce(1) para reduzir o DataFrame para 1 partição antes de salvar. Isso resulta em apenas 1 arquivo Parquet (útil para conjuntos de dados pequenos ou para evitar muitos arquivos pequenos)
produtos.coalesce(1).write.mode("overwrite").parquet("produtos_unico_arquivo_parquet")

In [20]:
# Usa repartition(10) para forçar 10 partições antes de salvar. Isso resulta em 10 arquivos Parquet, aumentando o paralelismo na leitura futura
produtos.repartition(10).write.mode("overwrite").parquet("produtos_part10_parquet")

In [21]:
# Salva o DataFrame e usa partitionBy("categoria_produto"). O Spark cria uma estrutura de pastas no disco
 #(ex: .../categoria_produto=mesa/..., .../categoria_produto=esporte/...), e os arquivos Parquet dentro dessas pastas refletem o particionamento interno do DataFrame
produtos.write.mode("overwrite").partitionBy("categoria_produto").parquet("produtos_part_categoria_parquet")

In [22]:
# Primeiro, o DataFrame é reduzido para 1 partição (coalesce(1)). Em seguida, ele é salvo com partitionBy("categoria_produto")
# Isso garante que haverá uma pasta por categoria, mas dentro de cada pasta, haverá apenas um arquivo Parquet (reduzindo arquivos pequenos)
produtos.coalesce(1).write.mode("overwrite").partitionBy("categoria_produto").parquet("produtos_unico_part_categoria_parquet")